# **HW2**


## **Handwritten Digits Dataset (scikit-learn)**
*   1797 instances
*   64 numerical features (intensity of 8x8 pixels)
*   10 classes (digits from 0 to 9)

![alt text](https://drive.google.com/uc?id=1JFgt0u6aIW8noh2saWgRTf9YtUTUTfbV)

1. Import scikit-learn libraries

In [3]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import sklearn

2. Load handwritten digits dataset (using load_digits()) and divide it into training (80%) and test (20%) datasets 

In [5]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# 데이터셋 로드
digits = load_digits()

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)


3. Import torch libraries

In [6]:
import torch

In [2]:
device = torch.cuda.is_available()
print(device)       # if GPU is available, else False

True


4. Prepare PyTorch Dataset and DataLoader for digits dataset

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

class DigitsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

# 데이터셋 로드
digits = load_digits()

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)

# 데이터셋 생성
train_dataset = DigitsDataset(X_train, y_train)
test_dataset = DigitsDataset(X_test, y_test)

# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


5. Define a simple network with one input layer, two hidden layers (HL1, HL2), and one output layer (OL)

*   H1: 32 neurons with ReLU activation
*   H2: 64 neurons with ReLU activation
*   OL: 10 neurons with Softmax activation


In [8]:
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()

        # 첫 번째 은닉층
        self.hl1 = nn.Linear(in_features=64, out_features=32)
        self.relu1 = nn.ReLU()

        # 두 번째 은닉층
        self.hl2 = nn.Linear(in_features=32, out_features=64)
        self.relu2 = nn.ReLU()

        # 출력층
        self.ol = nn.Linear(in_features=64, out_features=10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.hl1(x)
        x = self.relu1(x)
        x = self.hl2(x)
        x = self.relu2(x)
        x = self.ol(x)
        x = self.softmax(x)
        return x


6. Train the simple network with the following hyperparameters and settings:

*   Number of Epochs: 50
*   Learning rate: 0.01
*   Loss function: Cross Entropy
*   Optimization: Stochastic Gradient Descent



In [10]:
import torch.optim as optim

# 하이퍼파라미터
learning_rate = 0.01
num_epochs = 50

# 모델 초기화
model = SimpleNet()

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 훈련
for epoch in range(num_epochs):
    running_loss = 0.0
    for data, target in train_loader:
        optimizer.zero_grad() # 그래디언트 초기화
        outputs = model(data) # 예측값 계산
        loss = criterion(outputs, target) # 손실 함수 계산
        loss.backward() # 역전파
        optimizer.step() # 가중치 업데이트
        running_loss += loss.item()
    print("Epoch {} - Training loss: {:.4f}".format(epoch+1, running_loss/len(train_loader)))


Epoch 1 - Training loss: 2.2993
Epoch 2 - Training loss: 2.2799
Epoch 3 - Training loss: 2.2495
Epoch 4 - Training loss: 2.2010
Epoch 5 - Training loss: 2.1443
Epoch 6 - Training loss: 2.0757
Epoch 7 - Training loss: 2.0138
Epoch 8 - Training loss: 1.9735
Epoch 9 - Training loss: 1.9496
Epoch 10 - Training loss: 1.9332
Epoch 11 - Training loss: 1.9228
Epoch 12 - Training loss: 1.9135
Epoch 13 - Training loss: 1.9063
Epoch 14 - Training loss: 1.9006
Epoch 15 - Training loss: 1.8948
Epoch 16 - Training loss: 1.8899
Epoch 17 - Training loss: 1.8856
Epoch 18 - Training loss: 1.8823
Epoch 19 - Training loss: 1.8788
Epoch 20 - Training loss: 1.8758
Epoch 21 - Training loss: 1.8732
Epoch 22 - Training loss: 1.8709
Epoch 23 - Training loss: 1.8688
Epoch 24 - Training loss: 1.8664
Epoch 25 - Training loss: 1.8614
Epoch 26 - Training loss: 1.8377
Epoch 27 - Training loss: 1.8262
Epoch 28 - Training loss: 1.8177
Epoch 29 - Training loss: 1.8115
Epoch 30 - Training loss: 1.8073
Epoch 31 - Training

7. Evaluate the model performance on the test dataset using accuracy

In [11]:

correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print("Accuracy on test set: {:.2f}%".format(correct/total * 100))


Accuracy on test set: 72.78%


8. Train the simple network with the following hyperparameters and settings:

*   Number of Epochs: 50
*   Learning rate: 0.001
*   Loss function: Cross Entropy
*   Optimization: Adaptive Moment Estimation (Adam)

In [12]:
import torch.optim as optim

# 하이퍼파라미터
learning_rate = 0.001
num_epochs = 50

# 모델 초기화
model = SimpleNet()

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 훈련
for epoch in range(num_epochs):
    running_loss = 0.0
    for data, target in train_loader:
        optimizer.zero_grad() # 그래디언트 초기화
        outputs = model(data) # 예측값 계산
        loss = criterion(outputs, target) # 손실 함수 계산
        loss.backward() # 역전파
        optimizer.step() # 가중치 업데이트
        running_loss += loss.item()
    print("Epoch {} - Training loss: {:.4f}".format(epoch+1, running_loss/len(train_loader)))


Epoch 1 - Training loss: 2.1937
Epoch 2 - Training loss: 1.7750
Epoch 3 - Training loss: 1.5869
Epoch 4 - Training loss: 1.5446
Epoch 5 - Training loss: 1.5226
Epoch 6 - Training loss: 1.5116
Epoch 7 - Training loss: 1.5100
Epoch 8 - Training loss: 1.4972
Epoch 9 - Training loss: 1.4914
Epoch 10 - Training loss: 1.4929
Epoch 11 - Training loss: 1.4853
Epoch 12 - Training loss: 1.4820
Epoch 13 - Training loss: 1.4808
Epoch 14 - Training loss: 1.4831
Epoch 15 - Training loss: 1.4772
Epoch 16 - Training loss: 1.4751
Epoch 17 - Training loss: 1.4744
Epoch 18 - Training loss: 1.4733
Epoch 19 - Training loss: 1.4724
Epoch 20 - Training loss: 1.4720
Epoch 21 - Training loss: 1.4718
Epoch 22 - Training loss: 1.4712
Epoch 23 - Training loss: 1.4706
Epoch 24 - Training loss: 1.4706
Epoch 25 - Training loss: 1.4704
Epoch 26 - Training loss: 1.4698
Epoch 27 - Training loss: 1.4697
Epoch 28 - Training loss: 1.4694
Epoch 29 - Training loss: 1.4690
Epoch 30 - Training loss: 1.4690
Epoch 31 - Training